In [ ]:
  #APR = (1-p) / p / T
  #APR = T*(1-p) / p
  #APR = T*((1/p)-1)
  #APR/T = 1/p -1
  #APR/T + 1 = 1/p
  #p = 1/(APR*T + 1)


  #1/((2*y_reserves + x_reserves)/x_reserves)**t = 1/(APR*T + 1)
  #((2*y_reserves + x_reserves)/x_reserves)**t = (APR*T + 1)
  #((2*y_reserves + x_reserves)/x_reserves) = (APR*T + 1)**(1/t)
  #2*y_reserves/x_reserves + 1 = (APR*T + 1)**(1/t)
  #2*y_reserves/x_reserves = (APR*T + 1)**(1/t) - 1
  #y_reserves = x_reserves * ((APR*T + 1)**(1/t) - 1) / 2

def calc_y_reserves(APR,x_reserves,days_until_maturity,time_stretch):
    t=days_until_maturity/(365*time_stretch)
    T=days_until_maturity/365
    return x_reserves * ((T*APR/100 + 1)**(1/t) - 1) / 2

def calc_time_stretch(apr):
    return 3.09396 /( 0.02789 * apr)

def calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t):
    return 1/pow((y_reserves+total_supply)/x_reserves,t)

def calc_apr_from_spot_price(price,days_until_maturity):
    T=days_until_maturity/365
    return (1 - price) / price / T * 100

def add_liquidity(amount,x_reserves,y_reserves):
    #x/y = (x+amount)/(y+y_needed)
    #y+y_needed = (x+amount)/(x/y)
    #y_needed = (x+amount)/(x/y)-y
    y_needed = (x_reserves+amount)/(x_reserves/(y_reserves))-(y_reserves)
    return (x_reserves+amount,y_reserves+y_needed)

def calc_out_given_in(in_,in_reserves,out_reserves,token_out,g,t):
    k=pow(in_reserves,1-t) + pow(out_reserves,1-t)
    without_fee = out_reserves - pow(k-pow(in_reserves+in_,1-t),1/(1-t))
    if token_out == "base":
        fee =  (in_-without_fee)*g
        with_fee = without_fee-fee
    elif token_out == "pt":
        fee =  (without_fee-in_)*g
        with_fee = without_fee-fee
    return (with_fee,without_fee,fee)

def accrue_interest(APY,x_reserves,days_lapsed):
    return x_reserves * APY/100*days_lapsed/365

In [ ]:
###############################
# USER INPUTS
x_reserves = 1000 
initial_APR = 5 # Fixed Rate APR of PT
APR = initial_APR
APY = 5 # Yield Source APY
days_until_maturity = 365/2
#############################

print("Scenario: Yield Source APY {:,.2f}% Initial APR {:,.2f}%".format(APY,APR))

# pool setup
buffer = 0
time_stretch = calc_time_stretch(APR)
t=days_until_maturity/(365*time_stretch)
T=days_until_maturity/365
fee=0.1
y_reserves = calc_y_reserves(APR, x_reserves, days_until_maturity, time_stretch)
print("\ninitial reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
total_supply=x_reserves+y_reserves
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
print("initial 6 month apr: {:,.2f}".format(calc_apr_from_spot_price(spot_price,days_until_maturity)))

# new lp
lp_amount = 500
print("\nbob adds liquidity: {:,.2f} Base".format(lp_amount))
(x_reserves, y_reserves) = add_liquidity(lp_amount,x_reserves,y_reserves)
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
print("6 month apr: {:,.2f}".format(calc_apr_from_spot_price(spot_price,days_until_maturity)))

# fixed rate buy
buy_amount = 100
(with_fee,without_fee,fee)=calc_out_given_in(buy_amount,x_reserves,y_reserves+total_supply,"pt",fee,t)
print("\nalice buys: {:,.2f} PTs for {:,.2f} base".format(with_fee,buy_amount))
x_reserves+=buy_amount
y_reserves-=with_fee # NOTE: the fee is in terms of PT which means that less PT will have to be redeemed in 6 months
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))

# 6 months pass
target_apr = APY #Note: this target APR is set to that of the yield source, but could come from TWAR
x_reserves_from_interest = accrue_interest(APY,x_reserves,365/2)
y_reserves_from_interest = calc_y_reserves(target_apr, x_reserves_from_interest, days_until_maturity, time_stretch)
print("\nAdditional reserves from 6 months of interest with target APR of {:,.2F}%: {:,.2f} Base & {:,.2f} PT".format(target_apr,x_reserves_from_interest,y_reserves_from_interest))
x_reserves+=x_reserves_from_interest
y_reserves+=y_reserves_from_interest
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))

# redeem
redeem_amount = with_fee
print("\nalice redeems: {:,.2f} PTs for {:,.2f} Base".format(redeem_amount,redeem_amount))
x_reserves-=redeem_amount
y_reserves+=redeem_amount
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))



Scenario: Yield Source APY 5.00% Initial APR 5.00%

initial reserves: 1,000.00 Base & 995.64 PT
spot price: 0.9756097560975611
initial 6 month apr: 5.00

bob adds liquidity: 500.00 Base
new reserves: 1,500.00 Base & 1,493.47 PT
spot price: 0.9756097560975611
6 month apr: 5.00

alice buys: 102.16 PTs for 100.00 base
new reserves: 1,600.00 Base & 1,391.31 PT
spot price: 0.9775478273725636
6 month apr: 4.59

Additional reserves from 6 months of interest with target APR of 5.00%: 40.00 Base & 39.83 PT
new reserves: 1,640.00 Base & 1,431.13 PT
spot price: 0.977498421174467
6 month apr: 4.60

alice redeems: 102.16 PTs for 102.16 Base
new reserves: 1,537.84 Base & 1,533.29 PT
spot price: 0.9755892243070597
6 month apr: 5.00


In [ ]:
###############################
# USER INPUTS
x_reserves = 1000 
initial_APR = 5 # Fixed Rate APR of PT
APR = initial_APR
APY = 1 # Yield Source APY
days_until_maturity = 365/2
#############################

print("Scenario: Yield Source APY {:,.2f}% Initial APR {:,.2f}%".format(APY,APR))

# pool setup
buffer = 0
time_stretch = calc_time_stretch(APR)
t=days_until_maturity/(365*time_stretch)
T=days_until_maturity/365
fee=0.1
y_reserves = calc_y_reserves(APR, x_reserves, days_until_maturity, time_stretch)
print("\ninitial reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
total_supply=x_reserves+y_reserves
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
print("initial 6 month apr: {:,.2f}".format(calc_apr_from_spot_price(spot_price,days_until_maturity)))

# new lp
lp_amount = 500
print("\nbob adds liquidity: {:,.2f} Base".format(lp_amount))
(x_reserves, y_reserves) = add_liquidity(lp_amount,x_reserves,y_reserves)
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
print("6 month apr: {:,.2f}".format(calc_apr_from_spot_price(spot_price,days_until_maturity)))

# fixed rate buy
buy_amount = 100
(with_fee,without_fee,fee)=calc_out_given_in(buy_amount,x_reserves,y_reserves+total_supply,"pt",fee,t)
print("\nalice buys: {:,.2f} PTs for {:,.2f} base".format(with_fee,buy_amount))
x_reserves+=buy_amount
y_reserves-=with_fee # NOTE: the fee is in terms of PT which means that less PT will have to be redeemed in 6 months
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))

# 6 months pass
target_apr = APY #Note: this target APR is set to that of the yield source, but could come from TWAR
x_reserves_from_interest = accrue_interest(APY,x_reserves,365/2)
y_reserves_from_interest = calc_y_reserves(target_apr, x_reserves_from_interest, days_until_maturity, time_stretch)
print("\nAdditional reserves from 6 months of interest with target APR of {:,.2F}%: {:,.2f} Base & {:,.2f} PT".format(target_apr,x_reserves_from_interest,y_reserves_from_interest))
x_reserves+=x_reserves_from_interest
y_reserves+=y_reserves_from_interest
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))

# redeem
redeem_amount = with_fee
print("\nalice redeems: {:,.2f} PTs for {:,.2f} Base".format(redeem_amount,redeem_amount))
x_reserves-=redeem_amount
y_reserves+=redeem_amount
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))

Scenario: Yield Source APY 1.00% Initial APR 5.00%

initial reserves: 1,000.00 Base & 995.64 PT
spot price: 0.9756097560975611
initial 6 month apr: 5.00

bob adds liquidity: 500.00 Base
new reserves: 1,500.00 Base & 1,493.47 PT
spot price: 0.9756097560975611
6 month apr: 5.00

alice buys: 102.16 PTs for 100.00 base
new reserves: 1,600.00 Base & 1,391.31 PT
spot price: 0.9775478273725636
6 month apr: 4.59

Additional reserves from 6 months of interest with target APR of 1.00%: 8.00 Base & 0.99 PT
new reserves: 1,608.00 Base & 1,392.30 PT
spot price: 0.9776075864130808
6 month apr: 4.58

alice redeems: 102.16 PTs for 102.16 Base
new reserves: 1,505.84 Base & 1,494.46 PT
spot price: 0.975656913151981
6 month apr: 4.99


In [ ]:
###############################
# USER INPUTS
x_reserves = 1000 
initial_APR = 5 # Fixed Rate APR of PT
APR = initial_APR
APY = 10 # Yield Source APY
days_until_maturity = 365/2
#############################

print("Scenario: Yield Source APY {:,.2f}% Initial APR {:,.2f}%".format(APY,APR))

# pool setup
buffer = 0
time_stretch = calc_time_stretch(APR)
t=days_until_maturity/(365*time_stretch)
T=days_until_maturity/365
fee=0.1
y_reserves = calc_y_reserves(APR, x_reserves, days_until_maturity, time_stretch)
print("\ninitial reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
total_supply=x_reserves+y_reserves
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
print("initial 6 month apr: {:,.2f}".format(calc_apr_from_spot_price(spot_price,days_until_maturity)))

# new lp
lp_amount = 500
print("\nbob adds liquidity: {:,.2f} Base".format(lp_amount))
(x_reserves, y_reserves) = add_liquidity(lp_amount,x_reserves,y_reserves)
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
print("6 month apr: {:,.2f}".format(calc_apr_from_spot_price(spot_price,days_until_maturity)))

# fixed rate buy
buy_amount = 100
(with_fee,without_fee,fee)=calc_out_given_in(buy_amount,x_reserves,y_reserves+total_supply,"pt",fee,t)
print("\nalice buys: {:,.2f} PTs for {:,.2f} base".format(with_fee,buy_amount))
x_reserves+=buy_amount
y_reserves-=with_fee # NOTE: the fee is in terms of PT which means that less PT will have to be redeemed in 6 months
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))

# 6 months pass
target_apr = APY # Note: this target APR is set to that of the yield source, but could come from TWAR
x_reserves_from_interest = accrue_interest(APY,x_reserves,365/2)
y_reserves_from_interest = calc_y_reserves(target_apr, x_reserves_from_interest, days_until_maturity, time_stretch)
print("\nAdditional reserves from 6 months of interest with target APR of {:,.2F}%: {:,.2f} Base & {:,.2f} PT".format(target_apr,x_reserves_from_interest,y_reserves_from_interest))
x_reserves+=x_reserves_from_interest
y_reserves+=y_reserves_from_interest
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))

# redeem
redeem_amount = with_fee
print("\nalice redeems: {:,.2f} PTs for {:,.2f} Base".format(redeem_amount,redeem_amount))
x_reserves-=redeem_amount
y_reserves+=redeem_amount
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))


Scenario: Yield Source APY 10.00% Initial APR 5.00%

initial reserves: 1,000.00 Base & 995.64 PT
spot price: 0.9756097560975611
initial 6 month apr: 5.00

bob adds liquidity: 500.00 Base
new reserves: 1,500.00 Base & 1,493.47 PT
spot price: 0.9756097560975611
6 month apr: 5.00

alice buys: 102.16 PTs for 100.00 base
new reserves: 1,600.00 Base & 1,391.31 PT
spot price: 0.9775478273725636
6 month apr: 4.59

Additional reserves from 6 months of interest with target APR of 10.00%: 80.00 Base & 308.59 PT
new reserves: 1,680.00 Base & 1,699.89 PT
spot price: 0.9753729677806121
6 month apr: 5.05

alice redeems: 102.16 PTs for 102.16 Base
new reserves: 1,577.84 Base & 1,802.05 PT
spot price: 0.9735580040841973
6 month apr: 5.43


In [ ]:
## SHORT FIXED RATES ##

###############################
# USER INPUTS
x_reserves = 1000 
initial_APR = 5 # Fixed Rate APR of PT
APR = initial_APR
APY = 10 # Yield Source APY
days_until_maturity = 365/2
#############################
print("Scenario: Yield Source APY {:,.2f}% Initial APR {:,.2f}%".format(APY,APR))

# pool setup
buffer = 0
time_stretch = calc_time_stretch(APR)
t=days_until_maturity/(365*time_stretch)
T=days_until_maturity/365
fee=0.1
y_reserves = calc_y_reserves(APR, x_reserves, days_until_maturity, time_stretch)
print("\ninitial reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
total_supply=x_reserves+y_reserves
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
print("initial 6 month apr: {:,.2f}".format(calc_apr_from_spot_price(spot_price,days_until_maturity)))

# fixed rate buy
buy_amount = 100
(with_fee,without_fee,fee)=calc_out_given_in(buy_amount,x_reserves,y_reserves+total_supply,"pt",fee,t)
print("\nalice buys: {:,.2f} PTs for {:,.2f} base".format(with_fee,buy_amount))
x_reserves+=buy_amount
y_reserves-=with_fee # NOTE: the fee is in terms of PT which means that less PT will have to be redeemed in 6 months
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))

short_amount = 100
# sell short_amount of PT for base
(with_fee,without_fee,fee)=calc_out_given_in(short_amount,y_reserves+total_supply,x_reserves,"base",fee,t)
print("\nbob shorts: {:,.2f} PTs for {:,.2f} base".format(short_amount,with_fee))
x_reserves-=with_fee
y_reserves+=short_amount
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))
short_amount_base = with_fee

# 6 months pass
target_apr = APY #Note: this target APR is set to that of the yield source, but could come from TWAR
x_reserves_from_interest = accrue_interest(APY,x_reserves,365/2)
y_reserves_from_interest = calc_y_reserves(target_apr, x_reserves_from_interest, days_until_maturity, time_stretch)
print("\nAdditional reserves from 6 months of interest with target APR of {:,.2F}%: {:,.2f} Base & {:,.2f} PT".format(target_apr,x_reserves_from_interest,y_reserves_from_interest))
x_reserves+=x_reserves_from_interest
y_reserves+=y_reserves_from_interest
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))

# redeem
redeem_amount = with_fee
print("\nalice redeems: {:,.2f} PTs for {:,.2f} Base".format(redeem_amount,redeem_amount))
x_reserves-=redeem_amount
y_reserves+=redeem_amount
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))

# complete short by buying back short_amount of PT for base
(with_fee,without_fee,fee)=calc_out_given_in(short_amount_base,x_reserves,y_reserves+total_supply,"pt",fee,t)
print("\nbob completes short by purchasing: {:,.2f} PTs for {:,.2f} base".format(with_fee,short_amount_base))
x_reserves+=short_amount_base
y_reserves-=with_fee
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))



Scenario: Yield Source APY 10.00% Initial APR 5.00%

initial reserves: 1,000.00 Base & 995.64 PT
spot price: 0.9756097560975611
initial 6 month apr: 5.00

alice buys: 102.11 PTs for 100.00 base
new reserves: 1,100.00 Base & 893.53 PT
spot price: 0.9784892457449601
6 month apr: 4.40

bob shorts: 100.00 PTs for 97.17 base
new reserves: 1,002.83 Base & 993.53 PT
spot price: 0.9756821246171303
6 month apr: 4.98

Additional reserves from 6 months of interest with target APR of 10.00%: 50.14 Base & 193.41 PT
new reserves: 1,052.97 Base & 1,186.94 PT
spot price: 0.9737575458091902
6 month apr: 5.39

alice redeems: 97.17 PTs for 97.17 Base
new reserves: 955.80 Base & 1,284.11 PT
spot price: 0.9710230374396801
6 month apr: 5.97

bob completes short by purchasing: 98.45 PTs for 97.17 base
new reserves: 1,052.97 Base & 1,185.67 PT
spot price: 0.9737738739033822
6 month apr: 5.39


In [ ]:
###############################
# USER INPUTS
x_reserves = 0
initial_APR = 10
APR = initial_APR
APY = 10
days_until_maturity = 365/2
#############################
print("Scenario: Test Add/Remove LP Yield Source APY {:,.2f}% Initial APR {:,.2f}%".format(APY,APR))

# pool setup
buffer = 0
time_stretch = calc_time_stretch(APR)
t=days_until_maturity/(365*time_stretch)
T=days_until_maturity/365
fee=0.1

# add initial liquidity to pool
initial_lp_amount = 1
print("\nadd initial liquidity: {:,.2f} Base".format(initial_lp_amount))
x_reserves += initial_lp_amount
y_reserves = calc_y_reserves(initial_APR,x_reserves,days_until_maturity,time_stretch)
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))

# add lp
lp_amount = 1000
print("\nadd liquidity: {:,.2f} Base".format(lp_amount))
x_reserves += lp_amount
y_reserves = calc_y_reserves(APR,x_reserves,days_until_maturity,time_stretch)
total_supply=x_reserves+y_reserves
print("\nreserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))

# add lp
lp_amount = 100000
print("\nflash loan add liquidity: {:,.2f} Base".format(lp_amount))
#(x_reserves, y_reserves) = add_liquidity(lp_amount,x_reserves,y_reserves)
x_reserves += lp_amount
y_reserves = calc_y_reserves(initial_APR,x_reserves,days_until_maturity,time_stretch)
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("initial 6 month apr: {:,.2f}".format(APR))

# fixed rate buy
buy_amount = 957.19
(with_fee,without_fee,fee)=calc_out_given_in(buy_amount,x_reserves,y_reserves+total_supply,"pt",fee,t)
print("\nbuys: {:,.2f} PTs for {:,.2f} base".format(with_fee,buy_amount))
x_reserves+=buy_amount
y_reserves-=with_fee # NOTE: the fee is in terms of PT which means that less PT will have to be redeemed in 6 months
buffer+=with_fee # NOTE: the buffer ensures that the PTs are solvent
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))

# remove lp
withdrawal_amount = (x_reserves-buffer)*(lp_amount/101001)
print("\nflash loan remove liquidity: {:,.2f} Base".format(withdrawal_amount))
print("{:,.5f}".format(withdrawal_amount/lp_amount))
x_reserves -= withdrawal_amount
y_reserves = calc_y_reserves(APR,x_reserves,days_until_maturity,time_stretch)
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))

# remove lp
withdrawal_amount = (x_reserves-buffer) * (1000/1001)
print("\nremove liquidity: {:,.2f} Base".format(withdrawal_amount))
print("{:,.5f}".format(withdrawal_amount/1000))
x_reserves -= withdrawal_amount
y_reserves = calc_y_reserves(APR,x_reserves,days_until_maturity,time_stretch)
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))

# 6 months pass
target_apr = APY # Note: this target APR is set to that of the yield source, but could come from TWAR
x_reserves_from_interest = accrue_interest(APY,x_reserves,365/2)
y_reserves_from_interest = calc_y_reserves(target_apr, x_reserves_from_interest, days_until_maturity, time_stretch)
print("\nAdditional reserves from 6 months of interest with target APR of {:,.2F}%: {:,.2f} Base & {:,.2f} PT".format(target_apr,x_reserves_from_interest,y_reserves_from_interest))
x_reserves+=x_reserves_from_interest
y_reserves+=y_reserves_from_interest
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
spot_price=calc_spot_price_from_reserves(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
APR = calc_apr_from_spot_price(spot_price,days_until_maturity)
print("6 month apr: {:,.2f}".format(APR))

# sell/redeem pts
sell_amount = with_fee
(with_fee,without_fee,fee)=calc_out_given_in(sell_amount,y_reserves+total_supply,x_reserves,"base",fee,0)
print("\nsells: {:,.2f} PTs for {:,.2f} Base".format(sell_amount,with_fee))
x_reserves-=with_fee
y_reserves+=sell_amount
total_supply=x_reserves+y_reserves
buffer -= sell_amount
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))
print("buffer: {:}".format(buffer))

## Because there is leftover interest in the pool the address that initialized the pool should be able to claim it
withdrawal_amount = (x_reserves-buffer)*(initial_lp_amount/initial_lp_amount)
print("\nremove initial liquidity: {:,.2f} Base".format(withdrawal_amount))
x_reserves -= withdrawal_amount
y_reserves = calc_y_reserves(initial_APR,x_reserves,days_until_maturity,time_stretch)
total_supply=x_reserves+y_reserves
print("new reserves: {:,.2f} Base & {:,.2f} PT".format(x_reserves,y_reserves))

Scenario: Test Add/Remove LP Yield Source APY 10.00% Initial APR 10.00%

add initial liquidity: 1.00 Base
new reserves: 1.00 Base & 0.98 PT
spot price: 0.9523809523809523
6 month apr: 10.00

add liquidity: 1,000.00 Base

reserves: 1,001.00 Base & 977.00 PT
spot price: 0.9523809523809523
6 month apr: 10.00

flash loan add liquidity: 100,000.00 Base
new reserves: 101,001.00 Base & 98,579.73 PT
spot price: 0.9523809523809523
initial 6 month apr: 10.00

buys: 1,000.00 PTs for 957.19 base
new reserves: 101,958.19 Base & 97,579.73 PT
spot price: 0.9529363989606221
6 month apr: 9.88

flash loan remove liquidity: 99,957.61 Base
0.99958
new reserves: 2,000.58 Base & 1,914.67 PT
spot price: 0.9529363989606221
6 month apr: 9.88

remove liquidity: 999.58 Base
0.99958
new reserves: 1,001.00 Base & 958.02 PT
spot price: 0.9529363989606221
6 month apr: 9.88

Additional reserves from 6 months of interest with target APR of 10.00%: 50.05 Base & 48.85 PT
new reserves: 1,051.05 Base & 1,006.87 PT
spot pr

In [ ]:
# the relationship between the LP loss and fixed rate buys:
#1-(sum of all[(fixed rate purchase discount)/(new x reserves after trade)])

1-(1000-957.19)/101958.19

0.9995801220088352